In [3]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import KNNImputer
from pandas_profiling import ProfileReport

In [4]:
# path to database
my_path = os.path.join("..", "data", "datamining.db")

# connect to the database
conn = sqlite3.connect(my_path)

# the query
query = """
select
    age, 
    income, 
    frq, 
    rcn, 
    mnt, 
    clothes, 
    kitchen, 
    small_appliances, 
    toys, 
    house_keeping,
    dependents, 
    per_net_purchase,
    g.gender, 
    e.education, 
    m.status, 
    r.description
from customers as c
    join genders as g on g.id = c.gender_id
    join education_levels as e on e.id = c.education_id
    join marital_status as m on m.id = c.marital_status_id
    join recommendations as r on r.id = c.recommendation_id
order by c.id;
"""

df = pd.read_sql_query(query, conn)

In [5]:
non_metric_features = ["education", "status", "gender", "dependents", "description"]
metric_features = df.columns.drop(non_metric_features).to_list()

In [6]:
# copy dataset
df_central = df.copy()

- if you have more then 20% missing vals in a variable -> remove it (rule of thumb)

In [7]:
df_central.isna().sum()

age                 0
income              0
frq                 0
rcn                 0
mnt                 0
clothes             0
kitchen             0
small_appliances    0
toys                0
house_keeping       0
dependents          0
per_net_purchase    0
gender              0
education           0
status              0
description         0
dtype: int64

In [9]:
df_central.median()

C:\Users\hampe\Anaconda3\envs\datamining\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


age                 1966.0
frq                   17.0
rcn                   53.0
mnt                  383.0
clothes               51.0
kitchen                4.0
small_appliances      28.0
toys                   4.0
house_keeping          4.0
per_net_purchase      45.0
dtype: float64

In [10]:
# what central tendency method to use in case of missing vals in categorcial var? 
    -> mode 

IndentationError: unexpected indent (Temp/ipykernel_13572/2259413871.py, line 2)

## Fill missing values

- fill missing vals using median for numerical vars and using the mode for categroical and ordinal vars
    - concat the medians and means for each var -> df.fillna()
- other approach to fill missing vals: 
    - knn imputation -> can be used only for numerical vars (distance makes no sense with cat and ordinal vars)
- knn is a "lazy learner": no steps are needed to do the predicitons (it jsut needs the dataset) (e.g: naive bayes is not like this) 
- should we standadize data before applying knn? 
    -> yes!!!!
    - if A has vals around 1000 and B around 1: if i look for closest neighbors i basically just look for similar A vals...
- how to select the number of neighbors to use in knn method?
    - grid search -> evaluate how good a specific parameter combination is
- considering outliers when applying the knn???
    -e.g.: if var A has an outlier that has missing vals: if i look for its nearest neighbors, i am actually considering other outliers to fill in the missing vals of A
    -> remove outliers before applying knn!!
    
- only numerical var with missing values is income 
    - fill it with median / fill it using knn
        - sometimes it makes sense to simply use the median, but knowing what income means and having so much information about user profiles, it makes more sense to use knn

- should we use knn on binary (0/1) features -> No! (distance here can be calculated, but again doesnt make sense) 



## Remove Outliers
- plot histogram for each var -> which one do we think might has outliers? 
    - create a filter to only keep inliers (set the treshold manually for each variable (that we think does have outliers) based on its histogram)
    - kitchen, toys and housekeeping have very similar profiles 
        - if i consider outliers in one of them, i should be following the same approach with the others
- ideally how much of the dataset can we remove? what would be too much to remove? 
    - we may remove less then 3 percent (rule of thumb)
    - more than 5 is way too much
    - 2 would be great
    - 0 would be ideal (means we have no outliers) 
- rcn var: very long tail
    - removing the whole tail means removing too many outliers 
        - we could set a treshold: everthing above 100: we just set it to 100
        - also, looking at the scatterplots (pairwise correlatinos) we can see  that rcn has no correlations with others, it just seems to be random -> we'll probably not use this feature anyway later in the analysis 
            - but isnt the whole point to use uncorrelated vars? 
                - you wanna avoid completely correlated vars
                - you also dont wanna use completely uncorrelated vars
                    - vars describing the same phenomena from different aspects -> we expect them to have some correlation (but we dont want them two be 95% correlated, that would just mean they hold the exact same information)
- remove outliers with IQR method (inter quantile range) 
    - df.quantile(.25) -> Q1 
    - compute IQR value for each feature 
    - calculate lower and upper limit for each var 
    
    - apply it -> we see that it would remove 18% of the dataset 
        -> this is way too much -> adjus the IQR method: instead of 1.5, multiply with a larger value which would result in less outliers


- note: if we wanted to use knn for filling nans: the outlier neglection should be the first step!!
        
## Feature extraction

- homework: explore -> try to extract meaningful feature
    - manipulations of the current features that make more sense or can be useful (e.g: age instead of year of birth, avg amount spent per visit) 
    
                    
                    
                    